In [33]:
# Import the needed packages
import numpy as np
import pandas as pd
import bokeh.plotting as bk
import bokeh.io as bi
from bokeh.io import export_svgs
#bi.output_notebook()

# TUBS Rot, Gelb, Orange, Hellgrün, Grün, Dunkelgrün, Hellblau, Blau, Dunkelblau, Violett
TUBScolorscale=[(190,30,60,0.9),
               (255,200,42,0.9),
               (225,109,0,0.9),
               (172,193,58,0.9),
               (109,131,0,0.9),
               (0,83,74,0.9),
               (102,180,211,0.9),
               (0,112,155,0.9),
               (0,63,87,0.9),
               (138,48,127,0.9)]

In [34]:
# Open the data file
# Read the Data
Data = pd.read_csv('./Data/H01_TIRAND.csv', sep=';')
# Give the Columns
Data.columns

Index([u'Unnamed: 0', u'Degree', u'MS_RGA', u'w_MS_RGA', u'Grad_RGA', u'MS_A',
       u'w_MS_A', u'Grad_A', u'MS_D', u'w_MS_D', u'Grad_D', u'w_0.01_RGA',
       u'w_0.01_A', u'w_0.01_D', u'w_0.02_RGA', u'w_0.02_A', u'w_0.02_D',
       u'w_0.1_RGA', u'w_0.1_A', u'w_0.1_D', u'w_0.00666666666667_RGA',
       u'w_0.00666666666667_A', u'w_0.00666666666667_D', u'w_0.005_RGA',
       u'w_0.005_A', u'w_0.005_D'],
      dtype='object')

In [35]:
# Get the histogram of MS_RGA

from bokeh.layouts import gridplot, layout

sample_data = Data[['MS_RGA']]
x = np.linspace(1,2,100)
hist, edges = np.histogram(sample_data, density = True, bins = 1000, range=(1,2))

p1 = bk.figure(title="Distribution of Maximum Singular Values", y_axis_label = "Samples",height= 300, width = 800)
p1.quad(top=hist, bottom = 0, left=edges[:-1], right=edges[1:], alpha = 1, color = TUBScolorscale[6])
sample_data = Data[['MS_A']]

hist, edges = np.histogram(sample_data, density = True, bins = 1000, range=(1,2))
p2 = bk.figure(y_axis_label = "Samples",height= 300, width = 800)
p2.quad(top=hist, bottom = 0, left=edges[:-1], right=edges[1:], alpha = 1, color = TUBScolorscale[0])

hist, edges = np.histogram(sample_data, density = True, bins = 1000, range=(1,2))
p3 = bk.figure(y_axis_label = "Samples", x_axis_label = "Maximum Singular Value",height= 300, width = 800)
p3.quad(top=hist, bottom = 0, left=edges[:-1], right=edges[1:], alpha = 1, color = TUBScolorscale[2])

p = layout([[p1],[p2],[p3]])

bk.show(p)

In [36]:
# Other Plot
sample_data = Data[Data['MS_RGA']>0.0]['MS_RGA']
samples = range(0,len(sample_data))
p2 = bk.figure(title="Maximum Singular Values per Sample", x_axis_label = "Sample No.", y_axis_label="Maximum Singular Value",height= 800, width = 800, y_axis_type = "log")
p2.scatter(samples,sample_data, color = TUBScolorscale[6])
sample_data = Data[Data['MS_A']>0.0]['MS_A']
samples = range(0,len(sample_data))
p2.scatter(samples,sample_data, color = TUBScolorscale[0])
sample_data = Data[Data['MS_D']>0.0]['MS_D']
samples = range(0,len(sample_data))
p2.scatter(samples,sample_data, color = TUBScolorscale[2])
bk.show(p2)

In [37]:
# Other Plot
sample_data = Data[Data['MS_RGA']>0.0]['MS_RGA']
samples = Data[Data['MS_RGA']>0.0]['w_MS_RGA']
p2 = bk.figure(title="Maximum Singular Value over corresponding Frequency", x_axis_type = "log", x_axis_label ="Frequency of Maximum Singular Value", y_axis_label = "Maximum Singular Value",height= 800, width = 800,  y_axis_type = "log")
p2.scatter(samples,sample_data, color = TUBScolorscale[6])
sample_data = Data[Data['MS_A']>0.0]['MS_A']
samples = Data[Data['MS_RGA']>0.0]['w_MS_A']
p2.scatter(samples,sample_data, color = TUBScolorscale[0])
sample_data = Data[Data['MS_D']>0.0]['MS_D']
samples = Data[Data['MS_RGA']>0.0]['w_MS_D']
p2.scatter(samples,sample_data, color = TUBScolorscale[2])
bk.show(p2)